In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
from utils import k_fold_cross_validation_classification, acc_pre_rec

/Users/pinyen/opt/anaconda3/envs/da-ml/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
df_train = pd.read_csv('data/xgboost_distance_classification_prob_train')
df_test = pd.read_csv('data/xgboost_distance_classification_prob_test')


df_train = df_train[df_train['mode'] != np.nan]

df = pd.concat((df_train, df_test), ignore_index=True)
string_cols = ['survey_language', 'disability']
df[string_cols] = df[string_cols].astype('category')
df[string_cols] = df[string_cols].apply(lambda x: x.cat.codes)



In [4]:
df_train

,id,person_id,trip_n,travel_date,travel_date_dow,o_purpose_category,d_purpose_category,num_non_hh_travelers,num_hh_travelers,num_travelers,...,num_kids,num_workers,num_students,disability,trip_distance,mode,trip_distance_category,trip_distance_short_prob,trip_distance_medium_prob,trip_distance_long_prob
0,0,0,0,30/05/2019,4,7,2,0,1,1,...,2,1,2,none,13.4,subway,long,0.028715,0.053526,0.917759
1,1,0,1,01/06/2019,6,6,1,0,1,1,...,2,1,2,none,3.7,drive,long,0.194525,0.397987,0.407488
2,2,0,2,02/06/2019,7,7,7,1,2,3,...,2,1,2,none,9.8,drive,long,0.213565,0.378102,0.408333
3,3,0,3,02/06/2019,7,7,7,1,2,3,...,2,1,2,none,11.1,drive,long,0.213565,0.378102,0.408333
4,4,0,4,03/06/2019,1,7,7,0,1,1,...,2,1,2,none,2.5,drive,medium,0.340016,0.413909,0.246076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39439,39493,1794,53,07/06/2019,5,6,6,0,2,2,...,0,1,0,none,0.5,walk,short,0.582804,0.297657,0.119538
39440,39494,1794,54,07/06/2019,5,6,6,0,2,2,...,0,1,0,none,0.5,walk,short,0.582804,0.297657,0.119538
39441,39495,1794,55,07/06/2019,5,6,6,0,2,2,...,0,1,0,none,0.2,walk,short,0.582804,0.297657,0.119538
39442,39496,1794,56,07/06/2019,5,6,8,0,2,2,...,0,1,0,none,1.1,walk,medium,0.538093,0.302597,0.159310


In [5]:
TRANSPORT_MODES = ['drive', 'passenger', 'bus', 'subway', 'bike', 'walk', 'other']
candidate_cols = [
#      'travel_date_dow',
#      'o_purpose_category',
     'd_purpose_category',
     'num_non_hh_travelers',
#      'num_hh_travelers',
     'num_travelers',
#      'o_location_type',
#      'd_location_type',
     'o_congestion',
     'd_congestion',
#      'age',
#      'employment',
#      'student',
#      'license',
#      'planning_apps',
#      'industry',
#      'gender',
#      'education',
#      'survey_language',
#      'num_bicycles',
     'num_vehicles',
     'res_type',
#      'rent_own',
#      'income_aggregate',
#      'num_people',
#      'num_adults',
#      'num_kids',
#      'num_workers',
#      'num_students',
#      'disability'
#     'trip_distance',
    'trip_distance_short_prob',
    'trip_distance_medium_prob',
    'trip_distance_long_prob'
]

categorial_columns = ['travel_date_dow',
       'o_purpose_category', 'd_purpose_category', 'o_location_type',
       'd_location_type', 'age', 'employment', 'license', 'planning_apps', 'industry', 'gender'
                    , 'survey_language',
       'res_type', 'rent_own',  'disability']

onehot = pd.get_dummies(df[candidate_cols], columns=[x for x in categorial_columns if x in candidate_cols])

X_train_val = onehot.to_numpy()[:len(df_train)]
y_train_val = df['mode'][:len(df_train)].apply(lambda x: TRANSPORT_MODES.index(x))
group = df['person_id'].to_numpy()[:len(df_train)]

X_test = onehot.to_numpy()[len(df_train):]



XGBoost parameters: https://xgboost.readthedocs.io/en/stable/parameter.html

In [6]:
model = xgb.XGBClassifier(objective='multi:softprob',
                          n_estimators = 100,
                          n_jobs=-1,
                          num_class=7,
                          random_state=42,
                          eval_metric=['mlogloss', 'merror'],
                          use_label_encoder =False)


train_metrics, val_metrics = k_fold_cross_validation_classification(X_train_val, y_train_val, group, model)

In [7]:
train_metrics

[(0.6315639359847884, 0.9416982125297401),
 (0.635176675645698, 0.9341598879520175),
 (0.6388211060053874, 0.927895883150935),
 (0.6443352875930914, 0.9151491051555642),
 (0.6380719989859298, 0.9330175451665382)]

In [8]:
metrics = np.concatenate((train_metrics, val_metrics), axis=1)
metrics = metrics[:,[0, 2, 1, 3]]

df_metrics = pd.DataFrame(metrics, columns=['acc-train', 'acc-val', 'cross entropy-train', 'cros entropy-val'])

In [9]:
df_metrics

,acc-train,acc-val,cross entropy-train,cros entropy-val
0,0.631564,0.574598,0.941698,1.136939
1,0.635177,0.562936,0.934160,1.220186
2,0.638821,0.563189,0.927896,1.198797
3,0.644335,0.529725,0.915149,1.296148
4,0.638072,0.552865,0.933018,1.184225


## Parameter search

In [10]:
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV, GroupKFold

distributions = {
    'learning_rate': loguniform(10**-4, 1),
}

model = xgb.XGBClassifier(objective='multi:softprob',
                          n_estimators = 100,
                          n_jobs=4,
                          num_class=7,
                          random_state=42,
                          eval_metric=['mlogloss', 'merror'],
                          use_label_encoder =False)

def neg_log_loss(estimator, X, y):
    y_pred = estimator.predict_proba(X)
    print(-log_loss(y, y_pred))
    return -log_loss(y, y_pred)

k_fold = GroupKFold(n_splits=5)
search = RandomizedSearchCV(model,
                            distributions,
                            cv=k_fold,
                            scoring='neg_log_loss',
                            random_state=1,
                            n_jobs=4,
                            n_iter=20
                           ).fit(X_train_val, y_train_val, groups=group)

print('best parameter', search.best_params_)
print('best score', search.best_score_)


/Users/pinyen/opt/anaconda3/envs/da-ml/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/pinyen/opt/anaconda3/envs/da-ml/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/pinyen/opt/anaconda3/envs/da-ml/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/pinyen/opt/anaconda3/envs/da-ml/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated an

best parameter {'learning_rate': 0.05506529897945988}
best score -1.1437794797587473


In [11]:
search.cv_results_

{'mean_fit_time': array([29.53360972, 31.4361516 , 32.63969784, 31.80048542, 31.80072756,
        31.95987124, 32.09798398, 32.12544904, 32.62127662, 33.38728733,
        34.13237529, 34.16185617, 33.79377985, 31.06667085, 34.236268  ,
        33.52530828, 33.58916059, 34.07106066, 35.31015878, 34.15357633]),
 'std_fit_time': array([1.10680398, 0.47702197, 0.36125041, 0.1169873 , 0.40438678,
        0.32963035, 0.26257833, 0.26399307, 0.47588401, 0.35010389,
        0.54597674, 0.16685064, 0.8211521 , 0.27660427, 0.68154286,
        0.1559721 , 0.40296493, 0.60561158, 0.66630555, 1.66805205]),
 'mean_score_time': array([0.04287691, 0.04817863, 0.04554868, 0.04418831, 0.0469636 ,
        0.04533377, 0.04570761, 0.04482813, 0.04652901, 0.04656291,
        0.04649839, 0.05073004, 0.04783802, 0.05267715, 0.051265  ,
        0.05048618, 0.04828382, 0.05009952, 0.05207973, 0.04561291]),
 'std_score_time': array([0.00703152, 0.00200324, 0.00244287, 0.0015174 , 0.00566742,
        0.00214633, 

## Train on whole training/validation data, with tuned parameters.

In [12]:
# train on whole training-validation dataset
model = xgb.XGBClassifier(objective='multi:softprob',
                          n_estimators = 100,
                          learning_rate=0.055, # obtained from parameter search, takes too much time to rerun so I'll just hard-code it here.
                          n_jobs=-1,
                          num_class=7,
                          random_state=42,
                          eval_metric=['mlogloss', 'merror'],
                          use_label_encoder =False)


model.fit(X_train_val, y_train_val)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric=['mlogloss', 'merror'], gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.055, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_class=7, num_parallel_tree=1,
              objective='multi:softprob', predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False, ...)

In [13]:
y_test_prob = model.predict_proba(X_test)

In [14]:
y_test_prob

array([[0.04802497, 0.00512609, 0.08704595, ..., 0.01706883, 0.75014204,
        0.0093229 ],
       [0.04802497, 0.00512609, 0.08704595, ..., 0.01706883, 0.75014204,
        0.0093229 ],
       [0.08796829, 0.00724865, 0.09751408, ..., 0.02076613, 0.6143551 ,
        0.01681341],
       ...,
       [0.01342414, 0.01747837, 0.01264768, ..., 0.00444878, 0.9222363 ,
        0.00742382],
       [0.01342414, 0.01747837, 0.01264768, ..., 0.00444878, 0.9222363 ,
        0.00742382],
       [0.01628765, 0.01841126, 0.01669242, ..., 0.00909512, 0.8962816 ,
        0.00644125]], dtype=float32)

In [15]:
df_test_prob = pd.DataFrame(y_test_prob)
df_test_prob = pd.concat((df_test['id'], df_test_prob), axis=1, ignore_index=True)
df_test_prob.columns = ['id'] + TRANSPORT_MODES 

In [16]:
df_test_prob

,id,drive,passenger,bus,subway,bike,walk,other
0,39498,0.048025,0.005126,0.087046,0.083269,0.017069,0.750142,0.009323
1,39499,0.048025,0.005126,0.087046,0.083269,0.017069,0.750142,0.009323
2,39500,0.087968,0.007249,0.097514,0.155334,0.020766,0.614355,0.016813
3,39501,0.063746,0.006355,0.092687,0.106486,0.031983,0.687186,0.011558
4,39502,0.052509,0.005285,0.048704,0.085983,0.026596,0.771268,0.009655
...,...,...,...,...,...,...,...,...
26289,65787,0.013424,0.017478,0.012648,0.022341,0.004449,0.922236,0.007424
26290,65788,0.013424,0.017478,0.012648,0.022341,0.004449,0.922236,0.007424
26291,65789,0.013424,0.017478,0.012648,0.022341,0.004449,0.922236,0.007424
26292,65790,0.013424,0.017478,0.012648,0.022341,0.004449,0.922236,0.007424


In [17]:
df_test_prob.to_csv('prediction/xgboost_with_distance_classification_prob.csv', index=False)